## Multi-turn SQL Generation

In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/jupyter/git_repo/nl2sql-generic/nl2sql_src')
print(sys.path)

['/home/avijendra/nl2sql_lib/nl2sql-generic/notebooks', '/home/jupyter/git_repo/nl2sql-generic/nl2sql_src', '../', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/avijendra/fserv/lib/python3.11/site-packages']


In [2]:
import numpy as np
import faiss
from pandas import DataFrame
from datetime import datetime
from vertexai.preview.generative_models import GenerativeModel, GenerationResponse, Tool
from nl2sql_generic import Nl2sqlBq

from proto.marshal.collections import repeated
from proto.marshal.collections import maps


In [3]:
PROJECT_ID = 'sl-test-project-363109'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

In [4]:
# Initializing when metadata cache is already created
metadata_cache_file = "../nl2sql_src/cache_metadata/metadata_cache.json"
nl2sqlbq_client = Nl2sqlBq(project_id=PROJECT_ID, dataset_id=DATASET_ID, metadata_json_path = metadata_cache_file)

In [5]:
PGPROJ = "sl-test-project-353312"
PGLOCATION = 'us-central1'
PGINSTANCE = "test-nl2sql"
PGDB = "test-db"
PGTABLE = 'myqueries'
PGUSER = "postgres"
PGPWD = "test-nl2sql"
nl2sqlbq_client.init_pgdb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD, pg_table=PGTABLE)

PGSQL - class Initiated


In [6]:
def get_text(resp: GenerationResponse):
    part = resp.candidates[0].content.parts[0]
    try:
        text = part.text
    except:
        text = None
    return text

def prior_sql_result(question)->str:
    # Execute the SQL and return the result
    return "Test output for question : " + question


def call_api(name: str, args: str) -> str:
    if name == "prior_sql_tool":
        return prior_sql_result(args)


def recurse_proto_repeated_composite(repeated_object):
    repeated_list = []
    for item in repeated_object:
        if isinstance(item, repeated.RepeatedComposite):
            item = recurse_proto_repeated_composite(item)
            repeated_list.append(item)
        elif isinstance(item, maps.MapComposite):
            item = recurse_proto_marshal_to_dict(item)
            repeated_list.append(item)
        else:
            repeated_list.append(item)

    return repeated_list

def recurse_proto_marshal_to_dict(marshal_object):
    new_dict = {}
    for k, v in marshal_object.items():
      if not v:
        continue
      elif isinstance(v, maps.MapComposite):
          v = recurse_proto_marshal_to_dict(v)
      elif isinstance(v, repeated.RepeatedComposite):
          v = recurse_proto_repeated_composite(v)
      new_dict[k] = v

    return new_dict

previous_sql_spec = {
    "name": "prior_sql_tool",
    "description": "Provides the SQL query that is generated for the previous question",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The natural language question for which the SQL query was generated"
            }
        },
        "required": [
            "question"
        ]
    }
}

def get_function_name(response: GenerationResponse):
  return response.candidates[0].content.parts[0].function_call.name

def get_function_args(response: GenerationResponse) -> dict:
  return recurse_proto_marshal_to_dict(response.candidates[0].content.parts[0].function_call.args)


sql_tools = Tool.from_dict(
    {
        "function_declarations":[previous_sql_spec]
    }
)

In [7]:
# from vertexai.preview.generative_models import GenerativeModel
model = GenerativeModel("gemini-1.0-pro")

table_chat = model.start_chat()
sql_chat = model.start_chat()

In [8]:
def get_chat_response(chat_model, prompt) -> str:
    responses = chat_model.send_message(prompt, stream=True)
    output = []
    for response in responses:
        output.append(response.candidates[0].content.parts[0].text)
    return "".join(output)

In [9]:
questions = ["What is the total revenue generated by Marketing professionals?",
             "Which industry is the highest ?",
             "What is the average revenue?"
            ]

# questions = ["How many Black individuals are served across CalHHS programs?",
#              "What is the breakdown by program?",
#              "Has this changed over time?",
#              "Change over time by program?"
#             ]

question = questions[0]

In [10]:
table_prompt = nl2sqlbq_client.table_filter_promptonly("Table identification initiation")
# print(table_prompt)
table_chat.send_message(table_prompt)




candidates {
  content {
    role: "model"
    parts {
      text: "zi-companies-under-1000-employees-offering-tuition"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.122943789
    severity: HARM_SEVERITY_LOW
    severity_score: 0.334154755
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.183301061
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.144148842
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.173428148
    severity: HARM_SEVERITY_LOW
    severity_score: 0.243822455
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.172868907
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.112015955
  }
}
usage_metadata {
  prompt_token_count: 424
  candidates_token

In [11]:
q_prompt_template = """Using the context in the chat history, identify the table name that is most probable to contain the data requested for the question given below. 

Question: {question}
"""
for question in questions:
    q_prompt = q_prompt_template.format(question=question)
    table_identified = get_chat_response(table_chat, q_prompt)
    print("Table identified for queestion :'", question, "' is: ", table_identified)

Table identified for queestion :' What is the total revenue generated by Marketing professionals? ' is:  zi-companies-with-most-marketing-professionals
Table identified for queestion :' Which industry is the highest ? ' is:  zi-retail-companies-headquartered-california
Table identified for queestion :' What is the average revenue? ' is:  zi-retail-companies-headquartered-california


In [12]:
for question in questions:
    q_prompt = q_prompt_template.format(question=question)
    table_identified = get_chat_response(table_chat, q_prompt)
    table_identified = table_identified.strip()
    print("Table identified for queestion :'", question, "' is: ", table_identified)

    try:
        previous_question_sql = sql_chat.history[-1]
    except:
        previous_question_sql = ""

    sql_prompt = nl2sqlbq_client.generate_sql_few_shot_promptonly(question, table_name=table_identified, prev_sql=previous_question_sql)
    # print(sql_prompt)
    sql_gen = get_chat_response(sql_chat, sql_prompt)
    print("Generated SQL for question : ", question, " is \n ", sql_gen)




Table identified for queestion :' What is the total revenue generated by Marketing professionals? ' is:  zi-companies-with-most-marketing-professionals
Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata
Generated SQL for question :  What is the total revenue generated by Marketing professionals?  is 
  ```sql
SELECT zi_c_revenue as total_revenue
FROM `sl-test-project-363109.zoominfo.zi-companies-with-most-marketing-professionals`

```

Table identified for queestion :' Which industry is the highest ? ' is:  zi-retail-companies-headquartered-california
Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata
Generated SQL for question :  Which industry is the highest ?  is 
  ```sql
SELECT ZI_C_INDUSTRY_PRIMARY, SUM(ZI_C_REVENUE) AS total_revenue
FROM `sl-test-project-363109.zoominfo.zi-companies-with-most-marketing-professionals`
WHERE ZI_C_CITY = 'los angeles'
GROUP BY ZI_C_INDUSTRY_PRIMARY
ORDER BY to

['sl-test-project-353312:us-central1:test-nl2sql']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/home/avijendra/fserv/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/fserv/lib/python3.11/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno -3] Temporary failure in name resolution

The above exception was the direct cause of the following exception:

Traceback (most

In [15]:
question = "Which company has the highest revenue that is headquartered in California?"
table_identified = get_chat_response(table_chat, q_prompt)
previous_question_sql=""
sql_prompt = nl2sqlbq_client.generate_sql_few_shot_promptonly(question, table_name=table_identified, prev_sql=previous_question_sql)

sql_chat = model.start_chat()

sql_resp = sql_chat.send_message(sql_prompt, tools=[sql_tools])
txt = get_text(sql_resp)
if txt:
    print("Response from chat", txt)
else:
    fname = get_function_name(sql_resp)
    # fname = sql_resp.candidates[0].content.parts[0].function_call.name
    print("Function to call", fname)
    func_args = get_function_args(sql_resp)
    # fargs = sql_resp.candidates[0].content.parts[0].function_call.args
    print("function call arguments =", func_args)
    print(call_api(fname, func_args['question']))

Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata
Response from chat ```sql
SELECT
    company_name,
    headquarters_city,
    revenue
FROM
    `sl-test-project-363109.zoominfo.zi-retail-companies-headquartered-california` AS zc
WHERE
    zc.ZI_C_HEADQUARTERS = 1
ORDER BY
    revenue DESC
LIMIT 1
```


In [16]:
sql_chat.history

[role: "user"
 parts {
   text: "\nOnly use the following tables meta-data:\n\n```\nTable Name : zi-retail-companies-headquartered-california\n\nDescription: The table \'zi-retail-companies-headquartered-california\' contains information about retail companies headquartered in California. Each row in the table represents a different company, and the columns provide information such as the company name, headquarters city, number of employees, and annual revenue. The table is sorted by the number of employees, with the largest companies at the top.\n\nThis table has the following columns : \nZI_C_LOCATION_ID                     (INTEGER) : ZI_C_LOCATION_ID: Unique identifier for each location of the company.. Sample value for ZI_C_LOCATION_ID\nZI_C_COMPANY_ID                     (INTEGER) : ZI_C_COMPANY_ID: Unique identifier for the company.. Sample value for ZI_C_COMPANY_ID\nZI_C_IS_HQ                     (INTEGER) : ZI_C_IS_HQ: Flag indicating whether the location is the company\'s hea